# What's Covered?
- Rough overview of what the DatHDF object provides easy access to and what it can do
- Examples of accessing data in dat
- Examples of accessing logs in dat

# Standard Set-up of Examples
The standard set-up for these examples, you would not normally do this in every individual .ipynb file

In [74]:
import dat_analysis

def get_dat(datnum, raw=False, overwrite=False):
    """
    Define a simple get_dat function that knows where to load dats from and save them to

    Note: In a full setup where a path to `measurement-data` and a general save location have already been set in the dat_analysis config.toml, you can use:
        return dat_analysis.get_dat(datnum, host_name, user_name, experiment_name, raw=raw, overwrite=overwrite)
    """
    hdf_name = f"dat{datnum}{'_RAW' if raw else ''}.h5"
    return dat_analysis.get_dat_from_exp_filepath(f'experiment_dats/{hdf_name}', override_save_path=f'analyzed_dats/{hdf_name}', overwrite=overwrite)

# Load DatHDF object

In [75]:
dat = get_dat(6420, overwrite=True)  # Always start with a clean file for this example
print(type(dat))
dat

<class 'dat_analysis.dat.dat_hdf.DatHDF'>


Dat6420 - March 3, 2023 14:35:42

## Data Attr
Access to the experiment data is provided through the `.Data` attr of the `DatHDF` object

In [76]:
data_keys = dat.Data.data_keys
print(f'Data with these names are available in the HDF file: {data_keys}')

Data with these names are available in the HDF file: ['cscurrent_2d']


In [77]:
data = dat.Data.get_data('cscurrent_2d')
print(f"We've now loaded the data and we can see it has:\n\ttype: {type(data)}\n\tshape: {data.shape}")

We've now loaded the data and we can see it has:
	type: <class 'numpy.ndarray'>
	shape: (131, 5076)


In [78]:
x = dat.Data.x
y = dat.Data.y

print(f"x (and y) axes are always available by `.x` (and `.y`). \nThey have shapes:\n\tx: {x.shape}\n\ty: {y.shape}")

x (and y) axes are always available by `.x` (and `.y`). 
They have shapes:
	x: (5076,)
	y: (131,)


We can also save new data to the HDF easily

In [79]:
import numpy as np
dat.Data.set_data("new_data", np.linspace(0, 10, 100), subgroup='optional_subgroup')

In [80]:
print(f"We can see this new data has been added to the `data_keys`: {dat.Data.data_keys}")

We can see this new data has been added to the `data_keys`: ['cscurrent_2d', 'optional_subgroup/new_data']


In [81]:
new_data = dat.Data.get_data('optional_subgroup/new_data')
print(f"And we can load it like any other data and see that it has shape {new_data.shape} as expected")

And we can load it like any other data and see that it has shape (100,) as expected


## Logs Attr
Access to the experiment logs (sweeplogs, scan_vars, etc) is provided through the `.Logs` attr of the `DatHDF` object

### What's in the HDF

To see what exists directly in the HDF file (i.e. what was copied from the experiment HDF). 

You could then access these through the HDF directly (i.e. `with dat.hdf_read as f: ...`), but probably easier to use the shortcuts that are built into the `Logs` class

In [82]:
print(dat.Logs.logs_keys)

['Temperatures', 'sweep_logs_string', 'Magnet z', 'scan_vars_string', 'FastDAC1', 'FastDAC3', 'Other', 'General', 'FastDAC2']


### Shortcuts to commonly used logs

For more commonly used parts of sweeplogs there are shortcuts build in.

**Note: In jupyter, you can easily see what is accessible by pressing typing `dat.Logs.` then pressing `Tab`** 

#### Logs.sweeplogs

In [83]:
from pprint import pprint  # For nicer printing
sl = dat.Logs.sweeplogs
pprint(sl)

{'FastDAC 1': {'ADC0': 47.62136,
               'ADC1': 1164.12039,
               'ADC2': -0.80861,
               'ADC3': 2.58442,
               'AWG': {'AWG_used': 0,
                       'AW_Dacs': '2,0',
                       'AW_Waves': '0,1',
                       'measureFreq': 1269,
                       'numADCs': 2,
                       'numCycles': 1,
                       'numSteps': 127,
                       'numWaves': 2,
                       'samplingFreq': 2538.1,
                       'waveLen': 100},
               'DAC0{OHV*9950}': 0.0,
               'DAC1{CA2bias*10000}': -622.8638,
               'DAC2{OHC(10M)}': 0.0,
               'DAC3{RC2*122}': 0.0,
               'DAC4{P*2}': -419.3115,
               'DAC5{P*200}': 854.5183,
               'DAC6{CSQ2*1000}': 0.0,
               'DAC7{N*40}': 0.0,
               'MeasureFreq': 2538.1,
               'SamplingFreq': 2538.1,
               'visa_address': 'ASRL13::INSTR'},
 'FastDAC 2': {'ADC4'

As you can see, there is quite a lot of information saved in the sweeplogs... 

It's formed of nested dictionaries, to make accessing deeper parts of it, you can use the package `dictor` which is included in the `dat_analysis` installation

For example, to get to the `MC K` temperature which is quite nested, you can do:

In [84]:
from dictor import dictor

print(f"MC Temperatrue from sweeplogs = {dictor(sl, 'Lakeshore.Temperature.MC K')} K")

MC Temperatrue from sweeplogs = 0.0084655 K


#### Logs.temperatures

In [85]:
print(dat.Logs.temperatures)  # All stored in a NamedTuple
print(dat.Logs.temperatures.mc)  # Easy access to any individual temperature

Temperatures(fiftyk=62.6118, fourk=3.19588, magnet=3.78012, still=0.731352, mc=0.0084655)
0.0084655


#### Logs.dacs

In [86]:
dat.Logs.dacs

{'OHV*9950': 0.0,
 'CA2bias*10000': -622.8638,
 'OHC(10M)': 0.0,
 'RC2*122': 0.0,
 'P*2': -419.3115,
 'P*200': 854.5183,
 'CSQ2*1000': 0.0,
 'N*40': 0.0,
 'CSS': -339.9658,
 'N*2': -455.6274,
 'CSQ2*20': -3021.8506,
 'H1R': -744.9341,
 'RC2*11.1': -2299.8047,
 'CA1bias*113824': -4974.9756,
 'H2D': -1199.9512,
 'BD': -399.7803,
 'H1L': -899.9634,
 'H1C': -1449.8901,
 'HC': -1633.9111,
 'PINK': 9.7659,
 'CA2drain*3727': 186.1629,
 'CSS1': -199.8901,
 'DAC22': 0.0,
 'DAC23': 0.0}

#### Logs.scan_vars

These are the same ScanVars that are used in Igor during the scan.

In [87]:
dat.Logs.scan_vars

{'Filenum': 6420,
 'Traceback': 'mar3_BD_SD_varying_coupling_dottunes,Procedure,6556;DotTuneAround,Tim_Scans.ipf,163;ScanFastDAC2D,Scans.ipf,671;EndScan,ScanController.ipf,1739;SaveToHDF,ScanController_IO.ipf,335;addMetaFiles,ScanController_IO.ipf,82;',
 'x_label': 'P*200 (mV)',
 'y_label': 'PINK (mV)',
 'startx': -1145.5,
 'finx': 854.5,
 'numptsx': 5076,
 'channelsx': '5',
 'rampratex': 0,
 'delayx': 0,
 'is2D': 1,
 'starty': -90,
 'finy': 10,
 'numptsy': 131,
 'channelsy': '19',
 'rampratey': 0,
 'delayy': 0.01,
 'duration': 1.9999,
 'alternate': 0,
 'readVsTime': 0,
 'interlaced_y_flag': 0,
 'interlaced_channels': 'null',
 'interlaced_setpoints': 'null',
 'interlaced_num_setpoints': 0,
 'start_time': 3760698647.22,
 'end_time': 3760698942.63,
 'using_fastdac': 1,
 'comments': 'Dot Tuning, ',
 'numADCs': 1,
 'samplingFreq': 2538.1,
 'measureFreq': 2538.1,
 'sweeprate': 1000,
 'adcList': '1;',
 'startxs': '-1145.5',
 'finxs': '854.5',
 'startys': '-90',
 'finys': '10',
 'raw_wave_nam

#### Logs.get_fastdac()


In [88]:
pprint(dat.Logs.get_fastdac(1))

FastDAC(dac_vals={0: 0.0,
                  1: -622.8638,
                  2: 0.0,
                  3: 0.0,
                  4: -419.3115,
                  5: 854.5183,
                  6: 0.0,
                  7: 0.0},
        dac_names={0: 'OHV*9950',
                   1: 'CA2bias*10000',
                   2: 'OHC(10M)',
                   3: 'RC2*122',
                   4: 'P*2',
                   5: 'P*200',
                   6: 'CSQ2*1000',
                   7: 'N*40'},
        adcs={0: 47.62136, 1: 1164.12039, 2: -0.80861, 3: 2.58442},
        sample_freq=2538.1,
        measure_freq=2538.1,
        AWG={'AWG_used': 0,
             'AW_Dacs': '2,0',
             'AW_Waves': '0,1',
             'measureFreq': 1269,
             'numADCs': 2,
             'numCycles': 1,
             'numSteps': 127,
             'numWaves': 2,
             'samplingFreq': 2538.1,
             'waveLen': 100},
        visa_address='ASRL13::INSTR')


#### Other shortcuts

In [89]:
print('Comments:', dat.Logs.comments)
print('Time Completed:', dat.Logs.time_completed)
print('Magnets:', dat.Logs.magnets)
print('Measure Frequency (using FastDAC):', dat.Logs.measure_freq)
print('X axis label:', dat.Logs.x_label)
print('Y axis label:', dat.Logs.y_label)

Comments: ['Dot Tuning', '']
Time Completed: March 3, 2023 14:35:42
Magnets: Magnets(x=None, y=None, z=Magnet(axis='z', field=69.76, rate=50.157))
Measure Frequency (using FastDAC): 2538.07107
X axis label: P*200 (mV)
Y axis label: PINK (mV)
